In [1]:
import pandas as pd
import numpy as np
import datetime

In [6]:
df = pd.HDFStore('./metr-la.h5')['/df']
df

HDF5ExtError: HDF5 error back trace

  File "H5F.c", line 620, in H5Fopen
    unable to open file
  File "H5VLcallback.c", line 3501, in H5VL_file_open
    failed to iterate over available VOL connector plugins
  File "H5PLpath.c", line 578, in H5PL__path_table_iterate
    can't iterate over plugins in plugin path '(null)'
  File "H5PLpath.c", line 620, in H5PL__path_table_iterate_process_path
    can't open directory: /home/raoxuan/.conda/envs/STDM/lib/hdf5/plugin
  File "H5VLcallback.c", line 3351, in H5VL__file_open
    open failed
  File "H5VLnative_file.c", line 97, in H5VL__native_file_open
    unable to open file
  File "H5Fint.c", line 1898, in H5F_open
    unable to lock the file
  File "H5FD.c", line 1625, in H5FD_lock
    driver lock request failed
  File "H5FDsec2.c", line 1002, in H5FD__sec2_lock
    unable to lock file, errno = 11, error message = 'Resource temporarily unavailable'

End of HDF5 error back trace

Unable to open/create file './metr-la.h5'

In [3]:
timeslots = pd.date_range('2012-03-01 00:00:00', '2012-06-27 23:55:00', freq='5min')
print(len(timeslots))
days = pd.date_range('2012-03-01 00:00:00', '2012-06-27 23:55:00', freq='1D')
print(len(days), 0.8*len(days))
train_days = pd.date_range('2012-03-01 00:00:00', '2012-05-31 23:55:00', freq='1D')
print(len(train_days))
print(len(timeslots), timeslots[0], timeslots[-1])
list1 = [t.strftime('%Y-%m-%d %H:%M:%S')  for t in timeslots]
print(len(list1))
list2 = [pd.to_datetime(str(x)).strftime('%Y-%m-%d %H:%M:%S')  for x in df.index.values]
print(len(list2))
print(set(list1) - set(list2))

34272
119 95.2
92
34272 2012-03-01 00:00:00 2012-06-27 23:55:00
34272
34272
set()


In [4]:
data = df.stack().reset_index()
data.columns = ['timestamp', 'sensorid', 'speed']
data['timestamp'] = pd.to_datetime(data['timestamp'])
data['weekdaytime'] = data['timestamp'].dt.weekday * 288 + (data['timestamp'].dt.hour * 60 + data['timestamp'].dt.minute)//5
data

,timestamp,sensorid,speed,weekdaytime
0,2012-03-01 00:00:00,773869,64.375000,864
1,2012-03-01 00:00:00,767541,67.625000,864
2,2012-03-01 00:00:00,767542,67.125000,864
3,2012-03-01 00:00:00,717447,61.500000,864
4,2012-03-01 00:00:00,717446,66.875000,864
...,...,...,...,...
7094299,2012-06-27 23:55:00,717592,66.444444,863
7094300,2012-06-27 23:55:00,717595,68.444444,863
7094301,2012-06-27 23:55:00,772168,63.555556,863
7094302,2012-06-27 23:55:00,718141,68.666667,863


In [5]:
df_train = data[(data.timestamp >= '2012-03-01 00:00:00') & (data.timestamp <= '2012-05-31 23:55:00')]
df_train

,timestamp,sensorid,speed,weekdaytime
0,2012-03-01 00:00:00,773869,64.375000,864
1,2012-03-01 00:00:00,767541,67.625000,864
2,2012-03-01 00:00:00,767542,67.125000,864
3,2012-03-01 00:00:00,717447,61.500000,864
4,2012-03-01 00:00:00,717446,66.875000,864
...,...,...,...,...
5484667,2012-05-31 23:55:00,717592,63.222222,1151
5484668,2012-05-31 23:55:00,717595,62.333333,1151
5484669,2012-05-31 23:55:00,772168,62.777778,1151
5484670,2012-05-31 23:55:00,718141,69.000000,1151


In [6]:
df_train = df_train[['sensorid', 'weekdaytime', 'speed']]
df_train_avg = df_train.groupby(['sensorid', 'weekdaytime']).mean().reset_index()
df_train_avg

,sensorid,weekdaytime,speed
0,716328,0,61.656410
1,716328,1,61.751068
2,716328,2,61.470391
3,716328,3,56.248504
4,716328,4,61.320513
...,...,...,...
417307,774204,2011,54.870726
417308,774204,2012,55.675214
417309,774204,2013,54.159188
417310,774204,2014,54.730769


In [7]:
data_plus = pd.merge(data, df_train_avg, on=['sensorid', 'weekdaytime'], suffixes=(None, '_y'))
data_plus

,timestamp,sensorid,speed,weekdaytime,speed_y
0,2012-03-01 00:00:00,773869,64.375000,864,63.132937
1,2012-03-08 00:00:00,773869,66.777778,864,63.132937
2,2012-03-15 00:00:00,773869,66.750000,864,63.132937
3,2012-03-22 00:00:00,773869,59.666667,864,63.132937
4,2012-03-29 00:00:00,773869,64.444444,864,63.132937
...,...,...,...,...,...
7094299,2012-05-30 23:55:00,769373,62.444444,863,61.062118
7094300,2012-06-06 23:55:00,769373,58.444444,863,61.062118
7094301,2012-06-13 23:55:00,769373,63.500000,863,61.062118
7094302,2012-06-20 23:55:00,769373,61.250000,863,61.062118


In [8]:
sensorid_list = df.columns
timeslices = df.index
mux = pd.MultiIndex.from_product([timeslices, sensorid_list],names=['timestamp', 'sensorid'])
data_plus = data_plus.set_index(['timestamp', 'sensorid']).reindex(mux).reset_index()
data_plus['weekdaytime'] = data_plus['weekdaytime'] / data_plus['weekdaytime'].max()
data_plus

,timestamp,sensorid,speed,weekdaytime,speed_y
0,2012-03-01 00:00:00,773869,64.375000,0.428784,63.132937
1,2012-03-01 00:00:00,767541,67.625000,0.428784,64.867063
2,2012-03-01 00:00:00,767542,67.125000,0.428784,66.170635
3,2012-03-01 00:00:00,717447,61.500000,0.428784,61.943452
4,2012-03-01 00:00:00,717446,66.875000,0.428784,65.241071
...,...,...,...,...,...
7094299,2012-06-27 23:55:00,717592,66.444444,0.428288,62.377595
7094300,2012-06-27 23:55:00,717595,68.444444,0.428288,66.349054
7094301,2012-06-27 23:55:00,772168,63.555556,0.428288,64.207723
7094302,2012-06-27 23:55:00,718141,68.666667,0.428288,66.785409


In [9]:
np.array_equal(data_plus['speed'].values.reshape(-1, 207), df.values)

True

In [10]:
data_plus.to_csv(f'../METRLA/metr-la.csv.gz', index=False)
print('generate METR-LA PLUS over', data_plus.shape)

generate METR-LA PLUS over (7094304, 5)
